# List of Exposures with Hologram


- work with Weakly_2022_09
- use jupyter kernel LSST



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/03/22

- update : 2022/05/05
- last update : 2022/05/19 : check if 2022/05/04 where already ingested
- last update : 2022/09/13


In [1]:
import sys
print(sys.executable)

/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/bin/python


In [2]:
print(sys.path)

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/butlertools', '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python39.zip', '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9', '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/lib-dynload', '', '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages', '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages/Spectractor-2.1-py3.9.egg', '/sps/lsst/groups/auxtel/softs/utils/anaconda3_nov2022/lib/python3.9/site-packages/IPython/extensions', '/pbs/home/d/dagoret/.ipython']


In [3]:
! eups list -s | grep LOCAL

/bin/bash: eups: command not found


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


/bin/bash: eups: command not found


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

ModuleNotFoundError: No module named 'lsst'

In [ ]:
import lsst.daf.butler as dafButler

In [ ]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

## List of Exposures

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [ ]:
df_exposure

## Selection of science

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']

In [ ]:
df_science.sort_values(by="id",ascending=True,inplace=True)

In [ ]:
df_science.tail(60)

In [ ]:
df_science.day_obs.unique()

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [ ]:
date_sel = 20211103

In [ ]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [ ]:
len(df_science_selected)

# List of filters

In [ ]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

In [ ]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

## Selection of filter

- But now we have a loop on filters

In [ ]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

In [ ]:
len(all_def_science_selected_final)

In [ ]:
all_def_science_selected_final[0]

In [ ]:
all_def_science_selected_final[1]

In [ ]:
all_def_science_selected_final[2]

In [ ]:
all_def_science_selected_final[3]

In [ ]:
all_def_science_selected_final[4]